In [1]:
!pip install --quiet py_vncorenlp
!pip install --quiet pyvi

# Import Library

In [2]:
from datasets import load_dataset
import datasets
# from langdetect import detect, DetectorFactory
# from langdetect.lang_detect_exception import LangDetectException
from transformers import BertModel, BertTokenizer, PhobertTokenizer, AutoModel , AutoTokenizer , AutoModelForSequenceClassification
import torch
import torch.nn as nn
from pyvi.ViTokenizer import tokenize
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import DataCollatorWithPadding
import tqdm
import random
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report
import os
pd.set_option('display.max_colwidth', None)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random_seed = 42
torch.manual_seed(random_seed)
random_vector = torch.randn(5)
print(random_vector)

tensor([ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229])


# Config

In [4]:
VIETNAM_MODEL = 'vinai/phobert-base-v2'
ENGLISH_MODEL = 'cardiffnlp/twitter-roberta-base-sentiment'
DATA_PATH = "/kaggle/input/pho-mt-balance-label/phomt_balance_label.csv"
MAX_LEN = 50
BATCH_SIZE = 256
VIET_HIDDEN_DIM = 768

POS_DATA_SIZE = 50000
NEU_DATA_SIZE = 50000
NEG_DATA_SIZE = 50000

# 1. Data Loading

In [5]:
data = pd.read_csv (DATA_PATH)
data = data.sample (500000 , random_state = 42)

In [6]:
neg_data = data[data.label == 0].sample (NEG_DATA_SIZE, random_state = 42)
neu_data = data[data.label == 1].sample (NEU_DATA_SIZE, random_state = 42)
pos_data = data[data.label == 2].sample (POS_DATA_SIZE, random_state = 42)

balance_data = pd.concat ([neg_data, neu_data, pos_data])

In [7]:
balance_data.label.value_counts()

label
0    50000
1    50000
2    50000
Name: count, dtype: int64

In [8]:
balance_data

,Unnamed: 0,EnglishSentences,VietnameseSentences,label
314177,230557,"There is no life possible at the surface of Mars today , but it might still be hiding underground .\n","Ngày nay , không thể có sự sống trên bề mặt sau Hoả , nhưng nó có thể che dấu một thế giới ngầm .\n",0
178286,104045,They did n't want to send their books .\n,Họ không muốn gửi sách .\n,0
134765,965888,I do n't want to come .\n,Con không muốn đi .\n,0
124776,1198803,You will not believe what I ...\n,Chị sẽ không tin chuyện em\n,0
269284,2565456,"Although it was widely used in the past , the drug has mostly been discontinued and hence is now mostly no longer available .\n","Mặc dù nó đã được sử dụng rộng rãi trong quá khứ , loại thuốc này hầu hết đã bị ngưng sử dụng và do đó hiện nay hầu như không còn nữa .\n",0
...,...,...,...,...
340834,339690,"Drinking coconut water , which contains potassium and magnesium , can help with normal muscle function .\n","Uống nước dừa , Trong nước dừa có hàm lượng kali và ma-giê cao giúp điều hoà chức năng cơ bắp\n",2
261312,1223140,We 'll seize Munich .\n,Chúng ta sẽ chiếm lấy Munich .\n,2
437976,2169170,We may actually have a shot at finding the city .\n,Ta sẽ có cơ hội tìm ra thành phố đó .\n,2
253651,2021627,Respect 'em from the start .\n,Luật là phải tôn trọng .\n,2


# 2. Processing Data

In [9]:
id2label = {0: "NEGATIVE", 1 : "NEUTRAL" , 2: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 2 , "NEUTRAL" : 1}

vietnamese_tokenizer = AutoTokenizer.from_pretrained(VIETNAM_MODEL)
vietnamese_model = AutoModel.from_pretrained(VIETNAM_MODEL).to(device)

english_tokenizer = AutoTokenizer.from_pretrained(ENGLISH_MODEL)
english_model = AutoModelForSequenceClassification.from_pretrained(ENGLISH_MODEL, id2label = id2label, label2id = label2id).to(device)

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

## 2.1 Tokenize Data

In [10]:
# Tokenize sentence
def tokenize_sentences(english_sentences, vietnamese_sentences, vietnamese_tokenizer, english_tokenizer, seed=42):
    # Tokenize English sentences and generated positives/negatives
    english_sentences_tokenized = english_tokenizer(english_sentences, padding="max_length", max_length=MAX_LEN, truncation=True)

    # Tokenize Vietnamese sentences
    vietnamese_sentences_segmented = [tokenize(sentence) for sentence in vietnamese_sentences]
    vietnamese_sentences_tokenized = vietnamese_tokenizer(vietnamese_sentences_segmented, padding="max_length", max_length=MAX_LEN, truncation=True)
    
    return {
        "english_sentences_input_ids": english_sentences_tokenized["input_ids"],
        "english_sentences_attention_mask": english_sentences_tokenized["attention_mask"],
        "vietnamese_sentences_input_ids": vietnamese_sentences_tokenized["input_ids"], 
    }


def extract_matrice_embedding(input_ids, model , is_english = False):
    if is_english :
        with torch.no_grad():
            first_layer_embeddings = model.roberta.embeddings(input_ids.to(model.device))
    else :
        with torch.no_grad():
            first_layer_embeddings = model.embeddings(input_ids.to(model.device))
    return first_layer_embeddings.cpu()

In [11]:
translation_dataset_tokenized = tokenize_sentences(balance_data["EnglishSentences"].to_list(),
                                                   balance_data["VietnameseSentences"].to_list(),
                                                   vietnamese_tokenizer,
                                                   english_tokenizer)

## 2.2 Dataloader

In [12]:
# Custom dataset
class ContrastiveLearningDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset["english_sentences_input_ids"])

    def __getitem__(self, index):
        english_sentence_input_ids = torch.tensor(self.dataset["english_sentences_input_ids"][index], dtype = torch.long)
        english_sentence_attention_mask = torch.tensor(self.dataset["english_sentences_attention_mask"][index], dtype = torch.long)
        vietnamese_sentence_input_ids  = torch.tensor(self.dataset["vietnamese_sentences_input_ids"][index], dtype = torch.long)
        return {
            "english_sentence_input_ids": english_sentence_input_ids,
            "vietnamese_sentence_input_ids": vietnamese_sentence_input_ids,
            "english_sentence_attention_mask":english_sentence_attention_mask
        }

In [13]:
# Split dataset
def split_dataset(dataset):
    train_size = int(0.7 * len(dataset))  
    dev_size = int(0.15 * len(dataset))    
    test_size = len(dataset) - train_size - dev_size 
    torch.manual_seed(42)
    train_dataset, dev_dataset, test_dataset = random_split(dataset, [train_size, dev_size, test_size])
    return train_dataset, dev_dataset, test_dataset


In [14]:
custom_translation_dataset = ContrastiveLearningDataset(translation_dataset_tokenized)
train_dataset, val_dataset, test_dataset = split_dataset(custom_translation_dataset)

### Bug Checking

In [15]:
len(custom_translation_dataset) , len (train_dataset), len (val_dataset) , len(test_dataset)

(150000, 105000, 22500, 22500)

In [16]:
data = next(iter(custom_translation_dataset))
english_id = data["english_sentence_input_ids"]
vi_id = data["vietnamese_sentence_input_ids"]
att_mask_eng = data["english_sentence_attention_mask"]
print(vietnamese_tokenizer.decode (vi_id))
print (english_tokenizer.decode(english_id))
print (att_mask_eng)

<s> Ngày_nay , không_thể có sự sống trên bề_mặt sau Hoả , nhưng nó có_thể che dấu một thế_giới ngầm . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<s>There is no life possible at the surface of Mars today , but it might still be hiding underground .
</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])


## 2.3 extract ids, logits sentiment

In [17]:
# Custom collate function to handle padding within each batch
def collate_fn(batch):
    global english_tokenizer, vietnamese_tokenizer, english_model
    english_sentences_input_ids = [sample["english_sentence_input_ids"] for sample in batch]
    vietnamese_sentences_input_ids  = [sample["vietnamese_sentence_input_ids"] for sample in batch]
    
    english_sentences_attention_mask = [sample["english_sentence_attention_mask"] for sample in batch]
    # Padding to the maximum length in each batch using DataCollatorWithPadding from transformers
    # Ensure we pass the respective tokenizer for padding
    eng_padding_collator = DataCollatorWithPadding(tokenizer=english_tokenizer, padding=True, return_tensors='pt')
    vi_padding_collator = DataCollatorWithPadding(tokenizer=vietnamese_tokenizer, padding=True, return_tensors='pt')
    # collate the input_ids
    english_sentences_batch = eng_padding_collator({'input_ids': english_sentences_input_ids,"attention_mask" : english_sentences_attention_mask}).to(device)
    vietnamese_sentence_batch = vi_padding_collator({'input_ids': vietnamese_sentences_input_ids})

    
    vietnamese_embedding = extract_matrice_embedding(vietnamese_sentence_batch['input_ids'], vietnamese_model , is_english = False).to(device)
    
    with torch.no_grad():
            
            outputs = english_model(input_ids=english_sentences_batch['input_ids'], attention_mask=english_sentences_batch['attention_mask'])
            english_logits = outputs.logits
            
    
    return {
        'vietnamese_sentence_embeddings': vietnamese_embedding,
        'english_logits' : english_logits,

        # This is for the debug
        'english_id' : english_sentences_batch['input_ids'] ,
        'vi_id' : vietnamese_sentence_batch['input_ids']
    }

# Prepare the dataloader for training
def prepare_dataloader(dataset, batch_size):
    return DataLoader(dataset, batch_size = batch_size, shuffle = False, collate_fn=collate_fn)

In [18]:
train_dataloader = prepare_dataloader(train_dataset, BATCH_SIZE)
val_dataloader = prepare_dataloader(val_dataset, BATCH_SIZE)

### Bug checking

In [19]:
first_batch = next(iter(train_dataloader))

In [20]:
index = 11
english_id = first_batch["english_id"][index]
vi_id = first_batch["vi_id"][index]

print(vietnamese_tokenizer.decode (vi_id))
print (english_tokenizer.decode(english_id))
print (first_batch['english_logits'][index])

<s> tạo ra một lượng lớn ánh_sáng . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<s>This emits an enormous amount of light .
</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
tensor([-3.0736,  0.9439,  2.4364], device='cuda:0')


# 3. Model and Train

## 3.1 Define the mapper architecure

In [21]:
class MapperModel(nn.Module):
    def __init__(self, in_embedding_dim, out_embedding_dim, hidden_dim=256, num_heads=4, num_layers=1, dropout=0.1):
        """
        A more advanced model for mapping embeddings using transformer-based architecture.
        
        Args:
            in_embedding_dim (int): Dimension of input embeddings (Vietnamese embeddings).
            out_embedding_dim (int): Dimension of output embeddings (English embeddings).
            hidden_dim (int): Dimension of the hidden layers in the transformer.
            num_heads (int): Number of attention heads.
            num_layers (int): Number of transformer encoder layers.
            dropout (float): Dropout rate.
        """
        super(MapperModel, self).__init__()
        
        # Project input embeddings to hidden_dim
        self.input_projection = nn.Linear(in_embedding_dim, hidden_dim)
        self.input_layernorm = nn.LayerNorm(hidden_dim)
        
        # Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * 4,
            dropout=dropout,
            activation='relu'
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.encoder_layernorm = nn.LayerNorm(hidden_dim)
        
        # Output projection to map to out_embedding_dim
        self.output_projection = nn.Linear(hidden_dim, out_embedding_dim)
        self.output_layernorm = nn.LayerNorm(out_embedding_dim)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, vietnamese_embedding):
        """
        Forward pass for mapping embeddings.
        
        Args:
            vietnamese_embedding (Tensor): Input embeddings of shape (batch_size, seq_len, in_embedding_dim).
        
        Returns:
            Tensor: Output embeddings of shape (batch_size, seq_len, out_embedding_dim).
        """
        # Project input embeddings
        x = self.input_projection(vietnamese_embedding)  # (batch_size, seq_len, hidden_dim)
        x = self.input_layernorm(x)  # Normalize after projection
        
        # Prepare input for transformer (seq_len, batch_size, hidden_dim)
        x = x.permute(1, 0, 2)
        
        # Pass through transformer encoder
        x = self.transformer_encoder(x)  # (seq_len, batch_size, hidden_dim)
        x = self.encoder_layernorm(x)  # Normalize after encoder
        
        # Revert back to (batch_size, seq_len, hidden_dim)
        x = x.permute(1, 0, 2)
        
        # Project to output embedding space
        x = self.output_projection(x)  # (batch_size, seq_len, out_embedding_dim)
        x = self.output_layernorm(x)  # Normalize output
        
        # Apply dropout
        x = self.dropout(x)
        
        return x


## 3.2 For Loop For Training

In [22]:
import torch
from tqdm.auto import tqdm
import os

def train_mapper(model, optimizer, criterion, train_dataloader, val_dataloader, num_epochs=10, english_model=None, n_epochs_stop=5, save_path="best_model.pth"):
    """
    Train the Mapper model with early stopping and auto-saving.

    Args:
        model: The Mapper model to train.
        optimizer: Optimizer for training.
        criterion: Loss function.
        train_dataloader: DataLoader for training data.
        val_dataloader: DataLoader for validation data.
        num_epochs (int): Maximum number of epochs.
        english_model: Pretrained frozen English model.
        n_epochs_stop (int): Number of epochs to wait for improvement before stopping.
        save_path (str): Path to save the best model.
    """
    # Ensure the English model is frozen
    for param in english_model.parameters():
        param.requires_grad = False

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0.0

        # Training phase
        for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}"):
            vietnamese_embedding = batch['vietnamese_sentence_embeddings']
            english_logits = batch['english_logits']

            # Forward pass through the mapper
            mapped_embedding = model(vietnamese_embedding)

            # Forward pass through the frozen English model
            first_layer_embeddings = english_model.roberta.encoder(mapped_embedding)[0]
            fake_english_logits = english_model.classifier(first_layer_embeddings)

            # Compute loss
            loss = criterion(english_logits, fake_english_logits)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_dataloader)

        # Validation phase
        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validating"):
                vietnamese_embedding = batch['vietnamese_sentence_embeddings']
                english_logits = batch['english_logits']

                # Forward pass through the mapper
                mapped_embedding = model(vietnamese_embedding)

                # Forward pass through the frozen English model
                first_layer_embeddings = english_model.roberta.encoder(mapped_embedding)[0]
                fake_english_logits = english_model.classifier(first_layer_embeddings)

                # Compute validation loss
                val_loss = criterion(english_logits, fake_english_logits)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_dataloader)

        print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Early stopping and model saving
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Save the model
            torch.save(model.state_dict(), save_path)
            print(f"New best model saved with Val Loss: {best_val_loss:.4f}")
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= n_epochs_stop:
            print("Early stopping triggered. Training stopped.")
            break

    # Load the best model after training
    model.load_state_dict(torch.load(save_path))
    print(f"Training completed. Best Val Loss: {best_val_loss:.4f}. Model reloaded from {save_path}.")
    return model

In [23]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# Example usage:
model = MapperModel(768, 768).to(device)
model.apply(initialize_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [24]:
def count_parameters(model):
    """
    Count the total number of parameters in a PyTorch model.
    
    Args:
        model (nn.Module): The PyTorch model instance.
        
    Returns:
        int: The total number of parameters.
        int: The number of trainable parameters.
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

# Example usage:
total_params, trainable_params = count_parameters(model)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Total parameters: 1186560
Trainable parameters: 1186560


In [25]:
trained_mapper = train_mapper(
    model=model, 
    optimizer=optimizer,
    criterion = criterion,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    english_model=english_model,
    num_epochs=10,
    n_epochs_stop=3, save_path="/kaggle/working/best_model.pth"
)

Training Epoch 1/10:   0%|          | 0/411 [00:00<?, ?it/s]

Validating:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 1, Train Loss: 1.3973, Val Loss: 0.9556
New best model saved with Val Loss: 0.9556


Training Epoch 2/10:   0%|          | 0/411 [00:00<?, ?it/s]

Validating:   0%|          | 0/88 [00:00<?, ?it/s]

...

Epoch 9, Train Loss: 0.5816, Val Loss: 0.7000
New best model saved with Val Loss: 0.7000


Training Epoch 10/10:   0%|          | 0/411 [00:00<?, ?it/s]

Validating:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 10, Train Loss: 0.5625, Val Loss: 0.6995
New best model saved with Val Loss: 0.6995
Training completed. Best Val Loss: 0.6995. Model reloaded from /kaggle/working/best_model.pth.


/tmp/ipykernel_23/2300459941.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))


# 4. Testing Inference

In [26]:
english_model = AutoModelForSequenceClassification.from_pretrained(ENGLISH_MODEL, id2label = id2label, label2id = label2id).to(device)
vietnamese_model = AutoModel.from_pretrained(VIETNAM_MODEL).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
first_batch = next(iter(val_dataloader))

In [28]:
index = 20
english_logits = first_batch["english_logits"][index]
vi_id = first_batch["vi_id"][index]
english_id = first_batch['english_id'][index]

print ( "VietNam Sentence ----------- " , vietnamese_tokenizer.decode (vi_id) )
print ( "English Sentence ----------- " , english_tokenizer.decode (english_id) )

vietnamese_embeddings = extract_matrice_embedding(vi_id.view(1,-1), vietnamese_model ,  is_english = False).to(device)
# english_embeddings = extract_matrice_embedding(english_id.view(1,-1), english_model ,  is_english = True).to(device)
# Map Vietnamese embeddings to English space
with torch.no_grad () :
    mapped_vietnamese_embeddings = model(vietnamese_embeddings)

VietNam Sentence -----------  <s> Achondrit chiếm khoảng 8 % tổng_số thiên_thạch và phần_lớn ( khoảng 2 / 3 ) trong số chúng là thiên_thạch HED , có_thể bắt_nguồn từ lớp vỏ của tiểu hành_tinh 4 Vesta . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
English Sentence -----------  <s>Achondrites account for about 8 % of meteorites overall , and the majority ( about two thirds ) of them are HED meteorites , possibly originating from the crust of asteroid 4 Vesta .
</s><pad><pad><pad><pad><pad><pad>


In [29]:
encoder_outputs_fake = english_model.roberta.encoder(mapped_vietnamese_embeddings)[0] #
# # 6. Pass the CLS representation through the classification head to get logits
logits_fake = english_model.classifier(encoder_outputs_fake)
# print ( "Fake logits: " , logits_fake )
print ("Fake logits: " , logits_fake)
print ( "Real Logits: " , english_logits )

Fake logits:  tensor([[-1.4103,  1.8793, -0.6455]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
Real Logits:  tensor([-1.1557,  1.8930, -0.9384], device='cuda:0')


In [30]:
# F.cosine_similarity(encoder_outputs_fake, encoder_outputs_real, dim=-1).mean()

# 5. Mapper Sentiment Analysis

## 5.1 Evaluation Function

In [31]:
# Step 5: Use English Model for Sentiment Analysis
from sklearn.metrics import classification_report

def read_txt_to_data(file_path):
    """
    Reads the given TXT file and converts it into a list of dictionaries with 'text' and 'label'.
    """
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    text = []
    for line in lines:
        line = line.strip()
        if line in {"POS", "NEG", "NEU"}:
            # Map labels to indices
            label = {"NEG": 0, "NEU": 1, "POS": 2}[line]
            data.append({"text": " ".join(text).strip(), "label": label})
            text = []  # Reset text for the next entry
        else:
            text.append(line)
    return data

def sentiment_analysis(english_model, english_embeddings):
    # Pass the mapped embeddings through the English model for sentiment classification
    encoder_outputs = english_model.roberta.encoder (english_embeddings)[0]
    logits = english_model.classifier(encoder_outputs)
    #print (logits)
    predicted_class = torch.argmax(logits, dim=-1)
    return predicted_class.item()
    
def tokenize_vietnamese(text):
    # Tokenize the Vietnamese sentence using PhoBERT tokenizer
    vietname_segment = tokenize (text)
    tokens = vietnamese_tokenizer( vietname_segment , return_tensors='pt', truncation=True, padding=True, max_length = MAX_LEN)
    return tokens['input_ids']


# Step 5: Use English Model for Sentiment Analysis
def sentiment_analysis_2class (english_model, english_embeddings):
    """
    Perform sentiment analysis using the English model and handle 3-label output.
    Select between labels 0 (negative) and 2 (positive) based on logits.
    """
    # Pass the mapped embeddings through the English model for sentiment classification
    encoder_outputs = english_model.roberta.encoder(english_embeddings)[0]
    logits = english_model.classifier(encoder_outputs)
    
    # Extract logits for labels 0 and 2
    logits_0 = logits[:, 0]  # Logit for label 0 (negative)
    logits_2 = logits[:, 2]  # Logit for label 2 (positive)
    
    # Select the label with the higher logit
    predicted_class = 0 if logits_0 > logits_2 else 2
    return predicted_class


# Step 6: Sentiment Analysis Pipeline
def sentiment_analysis_pipeline(vietnamese_text, vietnamese_model, mapper, english_model, num_label = 3):
    # 1. Tokenize the Vietnamese text
    input_ids = tokenize_vietnamese(vietnamese_text)
    #print (input_ids)
    # 2. Extract Vietnamese embeddings (value and positional embeddings)
    vietnamese_embeddings = extract_matrice_embedding(input_ids, vietnamese_model, is_english = False).to(device)
    #print (vietnamese_embeddings)
    # 3. Map the Vietnamese embeddings to the English embedding space
    with torch.no_grad () :
        mapped_vietnamese_embeddings = mapper(vietnamese_embeddings)
    #print (mapped_vietnamese_embeddings)
    # 4. Perform sentiment analysis using the English model
    if num_label == 3 :
        sentiment_output = sentiment_analysis(english_model, mapped_vietnamese_embeddings)
    elif num_label == 2 :
        sentiment_output = sentiment_analysis_2class(english_model, mapped_vietnamese_embeddings)
    return sentiment_output


def reverse_labels(dataset):
    def map_label(example):
        if example['label'] == 0:
            return {'label': 2}  # Change positive (0) to 2
        elif example['label'] == 1:
            return {'label': 0}  # Change negative (1) to 0
        return example

    return dataset.map(map_label)



def evaluate_model(dataset, vietnamese_model, mapper, english_model, text_col , label_col, num_label = 3):
    """
    Evaluate the model on the given dataset.
    """
    true_labels = []
    predicted_labels = []

    for example in tqdm(dataset):
        vietnamese_text = example[text_col]
        true_label = example[label_col]

        # Predict sentiment using the pipeline
        predicted_label = sentiment_analysis_pipeline(vietnamese_text, vietnamese_model, mapper, english_model, num_label)

        true_labels.append(true_label)
        predicted_labels.append(predicted_label)
    if num_label == 3 :
        report = classification_report(true_labels, predicted_labels, target_names=["negative", "neutral", "positive"], digits=4)
    elif num_label == 2 :
        report = classification_report(true_labels, predicted_labels, target_names=["negative", "positive"], digits=4)
    return report

# Step 2: Convert to Hugging Face Dataset
def create_hf_dataset(data):
    """
    Converts a list of dictionaries into a Hugging Face dataset.
    """
    return datasets.Dataset.from_list(data)


def read_txt_files_to_dataset(folder_path, label = 2):
    """
    Reads all .txt files in a folder (and its subfolders), extracts text, and converts to Hugging Face Dataset.
    Each .txt file contains a single text line.
    
    Args:
        folder_path (str): Path to the root folder containing .txt files.
    
    Returns:
        datasets.Dataset: Hugging Face Dataset containing text and file path information.
    """
    data = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    text = f.read().strip()
                    data.append({"text": text, "file_path": file_path, "label" : label})
    
    # Convert to Hugging Face Dataset
    #hf_dataset = datasets.Dataset.from_list(data)
    return data

## 5.2 UIT VSFC

In [32]:
test1 = pd.read_csv ("/kaggle/input/translated-uit-vsfc-haiimphuong/translated_test_dataset_0.csv")
test2 = pd.read_csv ("/kaggle/input/translated-uit-vsfc-haiimphuong/translated_test_dataset_1.csv")
test3 = pd.read_csv ("/kaggle/input/translated-uit-vsfc-haiimphuong/translated_test_dataset_2.csv")
uit_vsfc_pandas = pd.concat ([test1,test2,test3])
uit_vsfc_hf = datasets.Dataset.from_pandas(uit_vsfc_pandas)
test_report = evaluate_model(uit_vsfc_hf, vietnamese_model, model, english_model, text_col = "sentences", label_col = "labels", num_label = 3 )
print("Test Dataset Evaluation Report:\n", test_report)

  0%|          | 0/3166 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

Test Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.8834    0.2903    0.4370      1409
     neutral     0.0854    0.6347    0.1506       167
    positive     0.8461    0.7780    0.8106      1590

    accuracy                         0.5534      3166
   macro avg     0.6050    0.5677    0.4660      3166
weighted avg     0.8226    0.5534    0.6095      3166



## 5.3 VIOCD BENCHMARK

In [33]:
from datasets import load_dataset
viocd_ds = load_dataset("tarudesu/ViOCD")
test_viocd = viocd_ds['test']
val_viocd = viocd_ds['validation']

README.md:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/200k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/197k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4387 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/548 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/549 [00:00<?, ? examples/s]

In [34]:
# Reverse the labels for test and validation datasets
test_viocd = reverse_labels(test_viocd)
val_viocd = reverse_labels(val_viocd)

# Evaluate on test and validation datasets
test_report = evaluate_model(test_viocd, vietnamese_model, model, english_model, text_col = "review", label_col = "label", num_label = 2)
val_report = evaluate_model(val_viocd, vietnamese_model, model, english_model, text_col = "review", label_col = "label", num_label = 2)

# Print the evaluation reports
print("Test Dataset Evaluation Report:\n", test_report)
print("Validation Dataset Evaluation Report:\n", val_report)

Map:   0%|          | 0/549 [00:00<?, ? examples/s]

Map:   0%|          | 0/548 [00:00<?, ? examples/s]

  0%|          | 0/549 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

Test Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.7762    0.5842    0.6667       279
    positive     0.6578    0.8259    0.7323       270

    accuracy                         0.7031       549
   macro avg     0.7170    0.7051    0.6995       549
weighted avg     0.7180    0.7031    0.6990       549

Validation Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.8065    0.6184    0.7000       283
    positive     0.6737    0.8415    0.7483       265

    accuracy                         0.7263       548
   macro avg     0.7401    0.7299    0.7242       548
weighted avg     0.7423    0.7263    0.7234       548



## 5.3 VLSP

In [35]:
# Main execution
file_path = "/kaggle/input/2016-vlsp/test_raw_ANS.txt"  # Update with your file path

# 1. Read and preprocess the TXT file
vlsp_data = read_txt_to_data(file_path)

# 2. Convert to Hugging Face Dataset
vlsp_dataset = create_hf_dataset(vlsp_data)

# 3. Perform Sentiment Analysis
#predictions = sentiment_analysis_on_dataset(hf_dataset, vietnamese_model, model, english_model)

val_report = evaluate_model(vlsp_dataset, vietnamese_model, model, english_model, text_col = "text", label_col = "label", num_label = 3)

# Print the evaluation reports
print("Test Dataset Evaluation Report:\n", test_report)

  0%|          | 0/1050 [00:00<?, ?it/s]

Test Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.7762    0.5842    0.6667       279
    positive     0.6578    0.8259    0.7323       270

    accuracy                         0.7031       549
   macro avg     0.7170    0.7051    0.6995       549
weighted avg     0.7180    0.7031    0.6990       549



## 5.4 AIVIVN 2019

In [36]:
import pandas as pd
import datasets
aivivn_dataset = pd.read_csv("/kaggle/input/aivivn/aivivn_test.csv")

In [37]:
aivivn_dataset_hf = datasets.Dataset.from_pandas(aivivn_dataset)
aivivn_dataset_hf = aivivn_dataset_hf.rename_columns ({"mapped_rating" : "label"})

In [38]:
reverse_aivivn = reverse_labels(aivivn_dataset_hf)
test_report = evaluate_model(reverse_aivivn, vietnamese_model, model, english_model, text_col = "discriptions", label_col = "label", num_label = 2)
# Print the evaluation reports
print("Test Dataset Evaluation Report:\n", test_report)

Map:   0%|          | 0/10094 [00:00<?, ? examples/s]

  0%|          | 0/10094 [00:00<?, ?it/s]

Test Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.8775    0.8394    0.8581      4796
    positive     0.8602    0.8939    0.8767      5298

    accuracy                         0.8680     10094
   macro avg     0.8688    0.8667    0.8674     10094
weighted avg     0.8684    0.8680    0.8678     10094



## 5.5 ntc-scv 

In [39]:
# Example usage
negative_folder_path = "/kaggle/input/ntc-scv/data_test/test/neg"  # Update with the path to your folder containing .txt files
negative_dataset = read_txt_files_to_dataset(negative_folder_path, label = 0)

# Example usage
positive_folder_path = "/kaggle/input/ntc-scv/data_test/test/pos"  # Update with the path to your folder containing .txt files
positive_dataset = read_txt_files_to_dataset(positive_folder_path, label = 2)

ntc_scv = pd.concat ([pd.DataFrame (negative_dataset), pd.DataFrame(positive_dataset)])
ntc_scv_dataset_hf = datasets.Dataset.from_pandas(ntc_scv)

In [40]:
test_report = evaluate_model(ntc_scv_dataset_hf, vietnamese_model, model, english_model, text_col = "text", label_col = "label", num_label = 2)
# Print the evaluation reports
print("Test Dataset Evaluation Report:\n", test_report)

  0%|          | 0/10000 [00:00<?, ?it/s]

Test Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.8099    0.4874    0.6086      5000
    positive     0.6334    0.8856    0.7386      5000

    accuracy                         0.6865     10000
   macro avg     0.7216    0.6865    0.6736     10000
weighted avg     0.7216    0.6865    0.6736     10000

